In [ ]:
# Load the PIL and numpy libraries
from PIL import Image
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import math
import heapq

# Read image from disk using PIL
occupancy_map_img = Image.open('D:\Mobile Robotics\occupancy_map.png')

# Interpret this image as a numpy array, and threshold its values to {0,1}
occupancy_grid = (np.asarray(occupancy_map_img) > 0).astype(int)

In [ ]:
#Implementation of A_star class

class A_star:
    def __init__(self, vertices, start, goal):
        self.vertices = vertices
        self.start = start
        self.goal = goal
        self.neighbours = [(-1,-1), (-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
        self.CostTo = defaultdict(lambda: float(np.inf))
        self.EstTotalCost = defaultdict(lambda: float(np.inf))
        self.pred = defaultdict(tuple)
        self.Q = []
        heapq.heappush(self.Q, (self.EstTotalCost[self.start], self.start))
        
        #Dictionary to keep track of the estimated total cost for the nodes, to update the priority queue 
        self.Q_check = {self.start: self.d(self.start,self.goal)}
    
    #function that performs the A_star search algorithm
    def A_star_search(self):
        self.CostTo[self.start] = 0
        self.EstTotalCost[self.start] = self.d(self.start,self.goal)
        
        while (self.Q != []):
            v = heapq.heappop(self.Q)[1]    

            if (v==self.goal):
                return(self.RecoverPath(), self.EstTotalCost[v])
                
            
            for i in self.N(v):
                
                #Calculating the cost to vertex i from vertex v without the heuristic 
                pvi = self.CostTo[v] + self.d(v,i)
                
                #Updating the predecessor dictionary with the vertex in the optimal path based on the least cost path 
                if (pvi<self.CostTo[i]):
                    self.pred[i] = v
                    self.CostTo[i] = pvi
                    #total cost to reach the vertex i including the heuristics
                    self.EstTotalCost[i] = self.CostTo[i] + self.d(i,self.goal)
                    
                    #Updating the priority for the vertex i in the priority queue, i.e, changing the cost to reach vertex i in the queue
                    if i in self.Q_check:
                        #remove the value from the dictionary and add the updated cost to the dictionary from the queue
                        val = self.Q_check[i]
                        self.Q.remove((val,i))
                        heapq.heappush(self.Q,(self.EstCostTo[i],i))
                        heapq.heapify(self.Q)
                        self.Q_check[i]=self.EstCostTo[i]    
                    #if the vertex isnt in the queue then add the value of the vertex to queue
                    else:
                        heapq.heappush(self.Q, (self.EstTotalCost[i], i))
                        
    #The function that implements the recover path from the start to goal vertex using the predecessor dictionary 
    def RecoverPath(self):
        curr_vertex = self.goal
        sequence = []
        while curr_vertex in self.pred:
            predecessor = self.pred[curr_vertex]
            sequence.append(predecessor)
            curr_vertex = predecessor
            if curr_vertex == self.start:
                break
        if curr_vertex != self.start:
            print('path not found')
            return(None)
        return sequence
    
    # Function to get the neighbors of vertex that are free in the occupancy grid
    def N(self,v):
        unocc_n = []
        for i in self.neighbours:
            if occupancy_grid[v[0]+i[0]][v[1]+i[1]] == 1:
                unocc_n.append((v[0]+i[0],v[1]+i[1]))
        return(unocc_n)
    
    #Function that calculates the distance between 2 vertices, gives the weight of reaching a vertex and the heuristics 
    def d(self,v1,v2):
        return(math.dist(v1,v2))

In [ ]:
#main function to run the A star algorithm

x = occupancy_grid.shape[0]
y = occupancy_grid.shape[1]
vertices = (x,y)
env = A_star(vertices, (635,140), (350,400))
OptimalPath, cost = env.A_star_search()

occupancy_grid[np.where(occupancy_grid > 0)] = 255
#Highlighting the optimal path in the occupancy grid map
for v in OptimalPath:
    occupancy_grid[v[0]][v[1]] = 105
    plt.plot(v[0], v[1], "go", markersize=0.5)
        
plt.imshow(np.transpose(occupancy_grid), cmap="Set1", origin="lower")
plt.legend(title = "cost = {}".format(cost))
plt.savefig('A_star path.png', dpi=500)